# Capstone Project: search for a location

### 1.1 A description of the problem and a discussion of the background

We want to open one coffee shop at 3 locations--Toronto (Canada), Chicago (US), Houston (US), and New York City (USA). All those three cities within the 2 to 3 million population range, except NYC--which is above 8 million. We want to find location options for one coffee shop in each city. We would like to open our coffee shop in the established coffee-lovers area with strong human traffic. Our approach isn't the optimal solution, however, it can be useful to get yourself familiar with potential areas for our business project. Also, the obtained data can be an optional backing base among others as data-driven support for protentional investors. If our data does not line up with expert's opinions on where to start our new coffee shop we could use our data as a good point to question any reals state expert recommendation. It is good to know what kind of data an expert has to make our data look less relevant.

Overall, the data from our project can be a starting point to find a local real state expert in the recommended areas.

Many companies are practicing remote "office" when workers stay and work from home. Due to decried demand for coffee at regular office locations, we can assume that that demand is redistributed among living areas--near staying at-home workers. Out model suppose to spot this "new normal". If so, that would be a good checking point for our model.

### 1.2 Description of the data and how it will be used to solve the problem

We would like to use the city's postal codes to scan for the most trending venues. Each postal code will help to split each city into parts, it is a good base for mapping and scanning for potential target--a good spot for a coffee shop. 

The postal codes can be obtained over the Internet. Geolocation of those ZIP codes can be sourced over the Geocoder Python module. Then those ZIP code coordinates can be used to obtain trending venues over Foursquare API. All the trending venues will be clustered. For clustering, we will use the DBSCAN clustering algorithm. We will look for clusters with trending Coffee Shop venues. Those clusters will be mapped with the Python Folium module. 
	
> Python modules:
- numpy
- pandas
 - folium
 - geocoder
 - foursquare
 - sklearn
 - matplotlib
	

	
> Postal Codes Sources:
- Toronto Postal Codes: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
- Chicago ZIP Codes: https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Chicago-Zip-Code-and-Neighborhood-Map/mapn-ahfc
- Houston: https://web.har.com/zipcode'
- New York City: https://www.nycbynatives.com/nyc_info/new_york_city_zip_codes.php
